<a href="https://colab.research.google.com/github/sakuna47/Fake_News_Detection/blob/main/Fake_News_Detection_Front_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Required Libraries

In [1]:
!pip install pandas numpy scikit-learn matplotlib seaborn nltk tensorflow streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.9 MB/s eta 0:00:00


Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Download NLTK stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Load and Explore the Dataset

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load real and fake news datasets
real_news = pd.read_csv('/content/drive/MyDrive/Fake News Detection/True.csv')
fake_news = pd.read_csv('/content/drive/MyDrive/Fake News Detection/Fake.csv')

# Add labels
real_news['label'] = 0  # 0 = Real
fake_news['label'] = 1  # 1 = Fake

# Combine datasets
data = pd.concat([real_news, fake_news], axis=0)
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the data

Mounted at /content/drive


Preprocess the Text Data

In [5]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = ''.join([c for c in text if c.isalpha() or c == ' '])
    # Remove stopwords and stem
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the 'text' column
data['cleaned_text'] = data['text'].apply(preprocess_text)

Feature Extraction (TF-IDF)

In [6]:
tfidf = TfidfVectorizer(max_features=5000)  # Use top 5000 features
X = tfidf.fit_transform(data['cleaned_text']).toarray()
y = data['label']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)